In [1]:
from tqdm import tqdm
from advsecurenet.models.model_factory import ModelFactory
from advsecurenet.datasets import DatasetFactory
from advsecurenet.dataloader import DataLoaderFactory
from advsecurenet.shared.types import DatasetType
from advsecurenet.utils.model_utils import train as train_model, test as test_model, save_model
from advsecurenet.shared.types.configs.train_config import TrainConfig
from advsecurenet.defenses import AdversarialTraining
from advsecurenet.attacks.fgsm import FGSM
from advsecurenet.attacks.pgd import PGD
from advsecurenet.shared.types.configs.defense_configs.adversarial_training_config import AdversarialTrainingConfig
import advsecurenet.shared.types.configs.attack_configs as AttackConfigs



/Users/melih/.pyenv/versions/3.10.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load basic MNIST model
mnist_model = ModelFactory.get_model(model_variant='CustomMnistModel', num_classes=10, num_input_channels=1)
mnist_model

custom module name advsecurenet.models.CustomModels.CustomMnistModel


CustomModel(
  (model): CustomMnistModel(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fc1): Linear(in_features=50176, out_features=512, bias=True)
    (relu): ReLU()
    (fc2): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [3]:
# load MNIST dataset
dataset = DatasetFactory.load_dataset(DatasetType.MNIST)
train_data = dataset.load_dataset(train=True)
test_data = dataset.load_dataset(train=False)
train_loader = DataLoaderFactory.get_dataloader(dataset=train_data, batch_size=128, shuffle=True)
test_loader = DataLoaderFactory.get_dataloader(dataset=test_data, batch_size=128, shuffle=False)
print(f"Train dataset size: {len(train_data)}")
print(f"Test dataset size: {len(test_data)}")

Train dataset size: 60000
Test dataset size: 10000


In [4]:
# first normal training
train_config = TrainConfig(model= mnist_model, train_loader=train_loader, epochs=5, device="mps")
train_model(train_config)
test_model(mnist_model, test_loader)

Training on mps


Epoch 1/5: 100%|██████████| 469/469 [00:46<00:00, 10.02it/s]


Epoch 1 - Average Loss: 0.152647


Epoch 2/5: 100%|██████████| 469/469 [00:43<00:00, 10.68it/s]


Epoch 2 - Average Loss: 0.033289


...

Epoch 5/5: 100%|██████████| 469/469 [00:44<00:00, 10.61it/s]


Epoch 5 - Average Loss: 0.008047
Training completed.
Testing on cpu


Testing: 100%|██████████| 79/79 [00:04<00:00, 16.11batch/s]


Test set: Average loss: 0.0004, Accuracy: 9868/10000 (98.68%)


(0.00038117431555874647, 98.68)

In [6]:
# Function to test model robustness against different attacks
def test_model_robustness(model, test_loader, attack, device):
    model.eval()  # Set the model to evaluation mode

    # Initialize counters
    correct = 0
    adv_correct = 0
    total = 0

    for data, target in tqdm(test_loader, desc='Testing'):
        # Send data and target to the same device as your model
        data, target = data.to(device), target.to(device)
        
        # Get the original model's predictions
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        
        # Generate adversarial data using the provided attack method
        fgsm_data = attack.attack(model=model, x=data, y=target)
        
        # Get the model's predictions on the adversarial data
        adv_output = model(fgsm_data)
        adv_pred = adv_output.argmax(dim=1, keepdim=True)
        adv_correct += adv_pred.eq(target.view_as(adv_pred)).sum().item()

        total += target.size(0)

    # Calculate the original accuracy
    original_accuracy = correct / total

    # Calculate the adversarial accuracy
    adversarial_accuracy = adv_correct / total

    # Calculate the robustness as the difference in accuracies
    robustness = original_accuracy - adversarial_accuracy

    print(f'Original Accuracy: {original_accuracy:.2%}')
    print(f'Adversarial Accuracy: {adversarial_accuracy:.2%}')
    print(f'Robustness (Accuracy Drop): {robustness:.2%}')


In [7]:
# get FGSM attack
fgsm_config = AttackConfigs.FgsmAttackConfig(epsilon=0.5, device="mps")
fgsm = FGSM(fgsm_config)

In [8]:
# Testing base model against FGSM attack 
test_model_robustness(mnist_model, test_loader, fgsm, device="mps")

Testing: 100%|██████████| 79/79 [00:11<00:00,  6.95it/s]

Original Accuracy: 98.68%
Adversarial Accuracy: 86.36%
Robustness (Accuracy Drop): 12.32%


In [9]:
# use fgsm to adversarially train the model
robust_model = ModelFactory.get_model(model_variant='CustomMnistModel', num_classes=10, num_input_channels=1)
adversarial_training_config = AdversarialTrainingConfig(model=robust_model, models=[robust_model], attacks=[fgsm], train_loader=train_loader, epochs=5, device="mps")
adversarial_training = AdversarialTraining(adversarial_training_config)
adversarial_training.adversarial_training()

custom module name advsecurenet.models.CustomModels.CustomMnistModel
Adversarial Training: Using mps for training


Epoch 1/5: 100%|██████████| 469/469 [01:26<00:00,  5.43it/s]


Epoch 1 - Average Loss: 0.158982


Epoch 2/5: 100%|██████████| 469/469 [01:27<00:00,  5.36it/s]


Epoch 2 - Average Loss: 0.026850


...

Epoch 3 - Average Loss: 0.017244


Epoch 4/5: 100%|██████████| 469/469 [01:26<00:00,  5.41it/s]


Epoch 4 - Average Loss: 0.017341


Epoch 5/5: 100%|██████████| 469/469 [01:26<00:00,  5.39it/s]


Epoch 5 - Average Loss: 0.025401
Adversarial Training: Training complete!


In [10]:
# testing the clean accuracy of the adversarially trained model
test_model(robust_model, test_loader)

Testing on cpu


Testing: 100%|██████████| 79/79 [00:04<00:00, 16.09batch/s]


Test set: Average loss: 0.0003, Accuracy: 9897/10000 (98.97%)


(0.000337365595212259, 98.97)